In [5]:
import pandas as pd
import textblob 

In [14]:
df = pd.read_csv("data.csv")
productDF = df[df["id"] == "AVphgVaX1cnluZ0-DR74"] #chosing a product
productDF.head(3)

C:\Users\janaa\AppData\Local\Temp\ipykernel_26304\2692392885.py:1: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data.csv")


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
3695,AVphgVaX1cnluZ0-DR74,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",B018Y229OU,Amazon,"Fire Tablets,Tablets,Computers & Tablets,All T...",firetablet7displaywifi8gbincludesspecialoffers...,Amazon,2016-12-23T00:00:00.000Z,2017-07-02T18:59:13Z,"2017-06-07T08:20:00.000Z,2017-04-30T00:09:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5025800/review...,"Good basic tablet for checking email , web bro...",Good,NaN,NaN,Jacques
3696,AVphgVaX1cnluZ0-DR74,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",B018Y229OU,Amazon,"Fire Tablets,Tablets,Computers & Tablets,All T...",firetablet7displaywifi8gbincludesspecialoffers...,Amazon,2016-12-23T00:00:00.000Z,2017-07-02T18:59:13Z,"2017-06-07T08:20:00.000Z,2017-04-30T00:09:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5025800/review...,I love this tablet. It was a great buy. I woul...,Great tablet!,NaN,NaN,resolution516
3697,AVphgVaX1cnluZ0-DR74,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",B018Y229OU,Amazon,"Fire Tablets,Tablets,Computers & Tablets,All T...",firetablet7displaywifi8gbincludesspecialoffers...,Amazon,2016-12-23T00:00:00.000Z,2017-07-02T18:59:13Z,"2017-06-07T08:20:00.000Z,2017-04-30T00:09:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5025800/review...,"Brought my old tablet in to my local store, an...",Good tablet for a low price.,NaN,NaN,AvatarRei
